In [3]:
import torch

import subprocess as sp

from tqdm.auto import tqdm
from glob import glob
from collections import defaultdict

from rdkit import Chem
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')

from utils import load_obj, save_obj

In [4]:
def get_ligand_mol(l_f):
    mol = Chem.MolFromMol2File(l_f)
    if mol == None:
        mol = Chem.SDMolSupplier(f'{l_f[:-5]}.sdf')[0]
    if mol == None:
        sp.run(f'obabel -imol2 {l_f} -omol2 -O {l_f[:-5]}_fix.mol2', shell=True, capture_output=True)
        mol = Chem.MolFromMol2File(f'{l_f[:-5]}_fix.mol2')
        sp.run(f'rm {l_f[:-5]}_fix.mol2', shell=True, capture_output=True)
    if mol == None:
        sp.run(f'obabel -imol2 {l_f} -osdf -O {l_f[:-5]}_fix.sdf', shell=True, capture_output=True)
        mol = Chem.SDMolSupplier(f'{l_f[:-5]}_fix.sdf')[0]
        sp.run(f'rm {l_f[:-5]}_fix.sdf', shell=True, capture_output=True)    
    if mol == None:
        sp.run(f'obabel -isdf {l_f[:-5]}.sdf -omol2 -O {l_f[:-5]}_fix.mol2', shell=True, capture_output=True)
        mol = Chem.MolFromMol2File(f'{l_f[:-5]}_fix.mol2')
        sp.run(f'rm {l_f[:-5]}_fix.mol2', shell=True, capture_output=True)
    if mol == None:
        sp.run(f'obabel -isdf {l_f[:-5]}.sdf -osdf -O {l_f[:-5]}_fix.sdf', shell=True, capture_output=True)
        mol = Chem.SDMolSupplier(f'{l_f[:-5]}_fix.sdf')[0]
        sp.run(f'rm {l_f[:-5]}_fix.sdf', shell=True, capture_output=True)
    return mol

In [7]:
refined_lig = [ file for file in  glob('/data/PDBbind/refined-set/*/*') if 'mol2' in file ]
other_lig = [ file for file in  glob('/data/PDBbind/v2020-other-PL/*/*') if 'mol2' in file ]
total_lig = refined_lig + other_lig

In [10]:
ligs = {}
for lig in tqdm( total_lig ):
    pdb = lig.split('/')[-2]
    try:
        mol = get_ligand_mol( lig )
        if mol != None:
            ligs[pdb] = mol
        else:
            pass
    except:
        pass
    # break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19446/19446 [00:20<00:00, 961.08it/s]


In [14]:
save_obj( ligs, '/data/PDBbind/ligand' )

In [17]:
refined_prot = [ file for file in glob("/data/PDBbind/refined-set/*/*") if 'protein.pdb' in file ]
other_prot   = [ file for file in glob("/data/PDBbind/v2020-other-PL/*/*") if 'protein.pdb' in file ]

total_prot = refined_prot + other_prot
proteins = { file.split("/")[-1][:4]: file for file in total_prot }

In [19]:
def get_protein_coordinate(pdb_f):
    return torch.tensor( [ [ float(line[30:38]), float(line[38:46]), float(line[46:54]) ] for line in open(pdb_f).readlines() if line[0:4] in ['ATOM', 'HETA'] and 'H' not in line[12:14] and 'HOH' not in line[17:20] ] )

def get_ligand_coordinate(mol):
    return mol.GetConformers()[0].GetPositions()

def calculate_atom_pair_distance(arr1, arr2):
    return torch.linalg.norm( arr1[:, None, :] - arr2[None, :, :], axis = -1)

def select_pocket_atom(pair_distance, cut_off):
    return torch.where( pair_distance < cut_off )[0]
    
def get_pocket_in_protein(pdb_f, select_index):
    total_atom = [ line for line in open(pdb_f).readlines() if line[0:4] in ['ATOM', 'HETA'] and 'H' not in line[12:14] and 'HOH' not in line[17:20] ]
    select_atom = [ line for idx, line in enumerate(total_atom) if idx in select_index ]

    select_residue = defaultdict(set)
    for idx, line in enumerate(total_atom):
        if idx in select_index:
            select_residue[line[21]].add( int(line[22:26]))
    
    total_lines = """"""
    for idx, line in enumerate(total_atom):
        if int( line[22:26] ) in select_residue[ line[21] ]:
            total_lines += line
    
    mol = Chem.MolFromPDBBlock( total_lines)
    Chem.AssignAtomChiralTagsFromStructure(mol)
    
    # a = mol.GetAtoms()
    # for atom in a:
    #     sym = atom.GetSymbol()
    #     if sym not in ['C', 'O', 'N','S', 'P', 'Se', 'F', 'Cl', 'Br', 'I', ]:
    #         print(sym, pdb_f)
    
    return mol

def get_pocket_residue(pdb_f, mol2_f):
    prot_xyz = get_protein_atom_xyz(pdb_f) 
    lig_xyz  = get_ligand_atom_xyz(mol2_f)
    
    pair_distance = calculate_atom_pair_distance(prot_xyz, lig_xyz)    
    select_prot_atom_index = select_pocket_atom(pair_distance, cut_off=8)
    
    return get_pocket_in_protein(pdb_f, select_prot_atom_index)

In [24]:
pocket = {}

for pdb, p_file in tqdm( proteins.items() ):
    try:
        protein_coord = get_protein_coordinate( p_file )
        ligand_coord  = get_ligand_coordinate( ligs[pdb] )
        pairwise_dist = calculate_atom_pair_distance( protein_coord, ligand_coord )

        selected_atom_index =  torch.where( pairwise_dist < 8 )[0] 
        pocket_mol = get_pocket_in_protein( p_file, selected_atom_index)    
        pocket[pdb] = pocket_mol
        
    except Exception as E:
        print(E, p_file)
    # break

  3%|█████▏                                                                                                                                                | 669/19443 [00:32<17:34, 17.81it/s]

Python argument types in
    rdkit.Chem.rdmolops.AssignAtomChiralTagsFromStructure(NoneType)
did not match C++ signature:
    AssignAtomChiralTagsFromStructure(RDKit::ROMol {lvalue} mol, int confId=-1, bool replaceExistingTags=True) /data/PDBbind/refined-set/1r1h/1r1h_protein.pdb


  4%|██████▋                                                                                                                                               | 859/19443 [00:41<13:04, 23.68it/s]

Python argument types in
    rdkit.Chem.rdmolops.AssignAtomChiralTagsFromStructure(NoneType)
did not match C++ signature:
    AssignAtomChiralTagsFromStructure(RDKit::ROMol {lvalue} mol, int confId=-1, bool replaceExistingTags=True) /data/PDBbind/refined-set/4abd/4abd_protein.pdb


 10%|███████████████▍                                                                                                                                     | 2016/19443 [01:39<13:23, 21.68it/s]

...

 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍     | 18712/19443 [16:35<00:36, 20.01it/s]

Python argument types in
    rdkit.Chem.rdmolops.AssignAtomChiralTagsFromStructure(NoneType)
did not match C++ signature:
    AssignAtomChiralTagsFromStructure(RDKit::ROMol {lvalue} mol, int confId=-1, bool replaceExistingTags=True) /data/PDBbind/v2020-other-PL/3gpe/3gpe_protein.pdb


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌    | 18865/19443 [16:43<00:27, 20.73it/s]

Python argument types in
    rdkit.Chem.rdmolops.AssignAtomChiralTagsFromStructure(NoneType)
did not match C++ signature:
    AssignAtomChiralTagsFromStructure(RDKit::ROMol {lvalue} mol, int confId=-1, bool replaceExistingTags=True) /data/PDBbind/v2020-other-PL/1f92/1f92_protein.pdb


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19443/19443 [17:16<00:00, 18.77it/s]


In [25]:
poc = {}
for i in pocket:
    if pocket[i] == None:
        pass
    else:
        poc[i] = pocket[i]

In [27]:
save_obj( poc, '/data/PDBbind/pocket' )

In [28]:
len(poc)

19391